In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:

# assuming the data is stored in a file called 'data.json'
with open('C:/Spacy/train_data.json') as f:
    cv_data = json.load(f)

print(cv_data[0])


['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [3]:
cv_data[1][1]

{'entities': [[2275, 2281, 'Companies worked at'],
  [2235, 2241, 'Companies worked at'],
  [1603, 1609, 'Companies worked at'],
  [667, 702, 'Skills'],
  [639, 657, 'College Name'],
  [612, 637, 'Degree'],
  [592, 610, 'College Name'],
  [587, 590, 'Degree'],
  [568, 574, 'Companies worked at'],
  [526, 536, 'Designation'],
  [515, 524, 'Location'],
  [507, 513, 'Companies worked at'],
  [491, 503, 'Designation'],
  [429, 438, 'Location'],
  [352, 361, 'Location'],
  [296, 305, 'Location'],
  [270, 279, 'Location'],
  [262, 268, 'Companies worked at'],
  [246, 258, 'Designation'],
  [238, 244, 'Companies worked at'],
  [226, 236, 'Designation'],
  [177, 207, 'Designation'],
  [150, 155, 'Years of Experience'],
  [54, 63, 'Location'],
  [43, 52, 'Location'],
  [35, 41, 'Companies worked at'],
  [19, 31, 'Designation'],
  [0, 18, 'Name']]}

In [4]:
!python -m spacy init fill-config "C:\Spacy\base_config.cfg" "C:\Spacy\config.cfg"


[+] Auto-filled config with all values
[+] Saved config
C:\Spacy\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
def get_spacy_doc(file,data):
    nlp = spacy.blank('en')
#     nlp = spacy.load("en_core_web_sm")
    db = DocBin()
    
    for text,annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        
        ents = []
        entity_indices = []
        
        for start,end,label in annot:
            skip_entity = False
            for idx in range(start,end):
                if idx in entity_indices:
                    skip_entity=True
                    break
                    
            if skip_entity==True:
                continue
                
            entity_indices = entity_indices + list(range(start,end))
            
            try:
                span =doc.char_span(start,end,label=label , alignment_mode='strict')
                
            except:
                continue
                
            if span is None:
                err_data = str([start,end]) + "   " +str(text) + "\n"
                file.write(err_data)
                
            else:
                ents.append(span)
                
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
        
    return db

In [14]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.3)

In [15]:
file = open('error.txt', 'w', encoding='utf-8', errors='ignore')


db = get_spacy_doc(file, train)
db.to_disk('train_data_2.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data_2.spacy')

file.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 68.78it/s]


In [18]:
# db = DocBin().from_disk('train_data.spacy', exclude=['vocab']).to_disk('train_data.spacy', encoding='utf-8')

In [16]:
!python -m spacy train "C:\Spacy\config.cfg" --output ./output --paths.train "C:\Users\hp\train_data.spacy" --paths.dev "C:\Users\hp\test_data.spacy"

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    164.04    0.00    0.00    0.00    0.00
  0      20        340.66   4725.68    0.00    0.00    0.00    0.00
  0      40         86.30    701.75    7.16   82.22    3.74    0.07
  0      60        432.13    627.09    9.67   77.27    5.16    0.10
  0      80        634.81   1580.92    9.27   72.06    4.95    0.09
  0     100        480.53    926.06   13.32   78.26    7.28    0.13
  0     120        244.68   1213.34   17.22   52.04   10.31    0.17
  1     140         49.61    536.10   17.73   77.34   10.01    0.18
  1     160         9

[2023-04-03 16:44:54,480] [INFO] Set up nlp object from config
[2023-04-03 16:44:54,500] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-03 16:44:54,507] [INFO] Created vocabulary
[2023-04-03 16:44:58,185] [INFO] Added vectors: en_core_web_lg
[2023-04-03 16:45:02,625] [INFO] Finished initializing nlp object
[2023-04-03 16:45:15,196] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### MOdel Test

In [17]:
nlp = spacy.load("C:/Users/hp/output/model-best")

In [18]:
doc = nlp('my name is nikhil mogre. I worked at microsoft. I have 10 years of experience')


In [19]:
doc

my name is nikhil mogre. I worked at microsoft. I have 10 years of experience

In [20]:
for ent in doc.ents:
    print(ent.text,"--->",ent.label_)

my name ---> Designation


### New Resume

In [21]:
# WHICH WILL READ PDF AND RETURN TEXT DATA 
!pip install pyMuPDF

     ---------------------------------------- 11.7/11.7 MB 1.9 MB/s eta 0:00:00


In [22]:
import sys,fitz

In [32]:
fname = "C:/Users/hp/Downloads/Alice Clark CV.pdf"

In [33]:
doc = fitz.open(fname)

In [34]:
doc

Document('C:/Users/hp/Downloads/Alice Clark CV.pdf')

In [35]:
# doc = [page.get_text() for page in doc]

In [36]:
text=" "
for page in doc:
    text = text + str(page.get_text())


In [37]:
text = text.strip()

In [38]:
text = " ".join(text.split())

In [39]:
text

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [30]:
doc

['Alice Clark \nAI / Machine Learning \n \nDelhi, India Email me on Indeed \n• \n20+ years of experience in data handling, design, and development \n• \nData Warehouse: Data analysis, star/snow flake scema data modelling and design specific to \ndata warehousing and business intelligence \n• \nDatabase: Experience in database designing, scalability, back-up and recovery, writing and \noptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, \nStream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake \nanalytics(U-SQL) \nWilling to relocate anywhere \n \nWORK EXPERIENCE \nSoftware Engineer \nMicrosoft – Bangalore, Karnataka \nJanuary 2000 to Present \n1. Microsoft Rewards Live dashboards: \nDescription: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping \nonline. Microsoft Rewards members can earn points when

In [40]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text,"--->",ent.label_)

Alice Clark ---> Name
AI / Machine Learning ---> Designation
Delhi ---> Location
Microsoft ---> Companies worked at
Software Engineer ---> Designation
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at


### My Resume

In [41]:
fname = "C:/Users/hp/Downloads/Nikhil-Mogre-Resume.pdf"

In [42]:
doc = fitz.open(fname)

In [43]:
doc

Document('C:/Users/hp/Downloads/Nikhil-Mogre-Resume.pdf')

In [44]:
text=" "
for page in doc:
    text = text + str(page.get_text())

In [45]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text,"--->",ent.label_)

Bachelor of Engineering(ME)
Job Analytics 
Playing chess ---> Degree
Data Analyst ---> Designation
SQL
nikhilmogre1998@gmail.com
linkedin.com/in/nikhilmogre/
+91-9175440381
 ---> Skills


### 3

In [46]:
fname = "C:/Users/hp/Downloads/Smith Resume.pdf"

In [47]:
doc = fitz.open(fname)

In [48]:
text=" "
for page in doc:
    text = text + str(page.get_text())

In [49]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text,"--->",ent.label_)

indeed.com/r/falicent/140749dace5dc26f ---> Email Address
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Manchester ---> Location
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
Microsoft ---> Companies worked at
